# Equity Linked Notes - Autocallable Phoenix Memory (basket)

This notebook demonstrates how to access and use the functionalities of **Autocallable Structured Notes** which are part of our **QPS** module within LSEG Financial Analytics SDK. This notebook presents a Phoenix Memory structure with basket underlying.

**You will be able to:**
* Define Phoenix Memory instrument with basic parameters
* Configure pricing parameters
* Evaluate Phoenix Memory Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp 

import json
import datetime as dt
import pandas as pd
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example Autocallable Phoenix Memory, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (strike, dates, notional, index)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing parameters, optional

In [2]:
# 1. Create SP definition object

SN_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "Phoenix_Memory_basket",
    inputs = [
        sp.NameTypeValue(name="notional", type = "string", value="1000"),
        sp.NameTypeValue(name="Basket", type = "string", value="TTEF_PA|IBE_MC|ENEI_MI"),
        sp.NameTypeValue(name="FirstAsset", type = "string", value="Perf_TTEF_PA"),
        sp.NameTypeValue(name="LastAsset", type = "string", value="Perf_ENEI_MI"),
        sp.NameTypeValue(name="BasketFunction", type = "string", value="WorstOf"),
        sp.NameTypeValue(name="BasketPerf", type = "string", value="If(\"BasketFunction\" == \"WorstOf\", Min(FirstAsset[t]:LastAsset[t]), IF(\"BasketFunction\" == \"Average\", Average(FirstAsset[t]:LastAsset[t],True), 0))"),
        sp.NameTypeValue(name="StrikeDate", type = "date", value="02/05/2025"),
        sp.NameTypeValue(name="MaturityDate", type = "date", value="16/05/2030"),
        sp.NameTypeValue(name="LastValuationDate", type = "date", value="02/05/2030"),
        sp.NameTypeValue(name="Schedule", type = "schedule", value=[
                    ["04/08/2025", "04/08/2025", "04/08/2025", "04/08/2025"],
                    ["03/11/2025", "03/11/2025", "03/11/2025", "03/11/2025"],
                    ["02/02/2026", "02/02/2026", "02/02/2026", "02/02/2026"],
                    ["04/05/2026", "04/05/2026", "04/05/2026", "04/05/2026"],
                    ["03/08/2026", "03/08/2026", "03/08/2026", "03/08/2026"],
                    ["02/11/2026", "02/11/2026", "02/11/2026", "02/11/2026"],
                    ["02/02/2027", "02/02/2027", "02/02/2027", "02/02/2027"],
                    ["03/05/2027", "03/05/2027", "03/05/2027", "03/05/2027"],
                    ["02/08/2027", "02/08/2027", "02/08/2027", "02/08/2027"],
                    ["02/11/2027", "02/11/2027", "02/11/2027", "02/11/2027"],
                    ["02/02/2028", "02/02/2028", "02/02/2028", "02/02/2028"],
                    ["02/05/2028", "02/05/2028", "02/05/2028", "02/05/2028"],
                    ["02/08/2028", "02/08/2028", "02/08/2028", "02/08/2028"],
                    ["02/11/2028", "02/11/2028", "02/11/2028", "02/11/2028"],
                    ["02/02/2029", "02/02/2029", "02/02/2029", "02/02/2029"],
                    ["02/05/2029", "02/05/2029", "02/05/2029", "02/05/2029"],
                    ["02/08/2029", "02/08/2029", "02/08/2029", "02/08/2029"],
                    ["02/11/2029", "02/11/2029", "02/11/2029", "02/11/2029"],
                    ["04/02/2030", "04/02/2030", "04/02/2030", "04/02/2030"]]),
        sp.NameTypeValue(name="CouponBarrier", type = "curve", value=[["02/05/2025","70%"]]),
        sp.NameTypeValue(name="FinalCouponBarrier", type = "string", value="70%"),
        sp.NameTypeValue(name="AutocallBarrier", type = "curve", value=[["02/05/2025","100%"]]),
        sp.NameTypeValue(name="RedemptionBarrier", type = "string", value="55%"),
        sp.NameTypeValue(name="CouponRate", type = "string", value="3.55%"),
        sp.NameTypeValue(name="Leverage", type = "string", value="100%"),
        sp.NameTypeValue(name="VarInit", type = "string", value="0"),
        sp.NameTypeValue(name="NbPeriodNoCallable", type = "string", value="3")
    ],
    payoff_description = [
          [
            "Schedule type",
            "Schedule description",
            "Repeat(Basket,#)",
            "Repeat(Basket,Perf_#)",
            "Performance",
            "Alive",
            "Early",
            "Count",
            "Coupon",
            "SumOfCoupons",
            "Settlement",
            "OptionAtMaturity",
            "PriceIn%",
            "Price"
          ],
          [
            "AtDate",
            "StrikeDate",
            "EqSpot(#)",
            "",
            "",
            "1",
            "",
            "$n=VarInit;$n",
            "",
            "0",
            "",
            "",
            "",
            ""
          ],
          [
            "OnUserSchedule",
            "Schedule",
            "EqSpot(#)",
            "#[t]/#[StrikeDate]",
            "BasketPerf",
            "If($n<NbPeriodNocallable,1,If(Performance[t]>=Interpol(AutocallBarrier,PS()),0,Alive[LastDate]))",
            "(1 - Alive[t]) * Alive[LastDate(-1)]",
            "$n=$n+1;$n",
            "Receive If(Performance[t]>=Interpol(CouponBarrier,PS()),$n*CouponRate-SumOfCoupons[LastDate(-1)],0)*Alive[LastDate(-1)]",
            "Coupon[t]+SumOfCoupons[LastDate]",
            "Receive Early[t]",
            "",
            "",
            ""
          ],
          [
            "AtDate",
            "LastValuationDate",
            "EqSpot(#)",
            "#[t]/#[StrikeDate]",
            "BasketPerf",
            "",
            "",
            "$n=$n+1;$n",
            "Receive (MaturityDate, If(Performance[t]>=FinalCouponBarrier, $n*CouponRate-SumOfCoupons[LastDate], 0) * Alive[LastDate])",
            "",
            "Receive (MaturityDate, Alive[LastDate])",
            "Receive (MaturityDate, If(Performance[t]>=RedemptionBarrier,0,(Performance[t] / Leverage -1)*Alive[LastDate]))",
            "Report((columnval(Coupon)+columnval(OptionAtMaturity)+columnval(Settlement))*100)",
            "Report(columnval(PriceIn%)/100*Notional)"
          ]
        ]
)


# 2. Create SP instrument definition object

phoenix_memory_basket = sp.StructuredProductsDefinitionInstrument(definition = SN_definition)
print("Instrument definition created")


# 3. Create SP parameters object - optional

phoenix_memory_basket_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2025, 10, 15),  # Set your desired valuation date
    report_ccy="EUR",  # Set your reporting currency
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
                  underlying_code = "TTEF.PA",
                  underlying_name = "TTEF_PA",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire"),
            sp.ModelDefinition(
                  underlying_code = "IBE.MC",
                  underlying_name = "IBE_MC",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire"),
            sp.ModelDefinition(
                  underlying_code = "ENEI.MI",
                  underlying_name = "ENEI_MI",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire")
          ]
)

print("Pricing parameters configured")

Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

try:
    response = sp.price(
        definitions=[phoenix_memory_basket],
        pricing_preferences=phoenix_memory_basket_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )

    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
    
except Exception as e:
    print(f"Price calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike, dates, notional, underlying, barrier, profit target), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` (payments, fixings)
   - **description**: Instrument summary StructuredProductDefinition and also the default fields not specified in the StructuredProductDefinition, but used by default in the calculation
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

#### Description
Useful for understanding which fields are included by default in the price function, even if they are not explicitly specified.

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "Phoenix_Memory_basket",
    "dealCcy": "EUR",
    "reportCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 10.964544953,
        "Count": 196.792459929,
        "Coupon": 0.276337406,
        "ENEI_MI": 138.982515606,
        "Early": 0.447733152,
        "IBE_MC": 288.00011241,
        "OptionAtMaturity": -0.220577641,
        "PERF_ENEI_MI": 18.106111986,
        "PERF_IBE_MC": 18.561232574,
        "PERF_TTEF_PA": 16.648137272,
        "Performance": 13.316475588,
        "Price": 997.352812858,
        "PriceIn%": 99.735281286,
        "Settlement": 0.941593048,
        "SumOfCoupons": 3.870057026,
        "TTEF_PA": 846.724261648
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, 

#### Analytics

##### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_phoenix_memory_basket_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_phoenix_memory_basket_valuation)

,Field,Value
0,marketValueInDealCcy,997.352813
1,marketValueInReportCcy,997.352813


##### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Extract underlyings
model_df = pd.DataFrame(data=phoenix_memory_basket_pricing_params.models)
underlying_list = model_df['underlyingName'].to_list()

# Build dataframes for all cash flow types
output = []
for cf_type in cashflows:
    cashflow_df = pd.DataFrame(cf_type['payments'])
    if cf_type['legTag'] == 'Index':
        cashflow_df = cashflow_df.rename(columns={'amount': underlying_list[0]})
    else:
        cashflow_df = cashflow_df.rename(columns={'amount': cf_type['legTag']})
    cashflow_df['discountFactor'] = cashflow_df['discountFactor'].round(4)
    output.append(cashflow_df)

# Merge all dataframes on the 'date' column
combined_df = pd.concat([*output],axis=1)

# Remove duplicated columns with the same values
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

common_cols = ['date', 'discountFactor', 'Performance', 'Alive', 'Early', 'Coupon', 'Settlement', 'OptionAtMaturity', 'Price', 'currency', 'occurence']
indv_perf = [f'PERF_{underlying}' for underlying in underlying_list if len(underlying_list) > 1]
cols_to_display = common_cols[0:2] + underlying_list + indv_perf + common_cols[2:]

# Leave only columns to display
combined_df = combined_df.loc[:,[*cols_to_display]]

# Display the combined dataframe
display(combined_df)

,date,discountFactor,TTEF_PA,IBE_MC,ENEI_MI,PERF_TTEF_PA,PERF_IBE_MC,PERF_ENEI_MI,Performance,Alive,Early,Coupon,Settlement,OptionAtMaturity,Price,currency,occurence
0,2025-05-02,1.0000,50.860000,15.516217,7.676000,0.000000,0.000000,0.000000,0.000000,1.0000,0.0000,0.000000,0.000000,0.000000,0.000000,EUR,Historical
1,2025-08-04,1.0000,51.580000,15.500000,7.854000,1.014157,0.998955,1.023189,0.998955,1.0000,0.0000,0.035500,0.000000,0.000000,0.000000,EUR,Historical
2,2025-11-03,0.9989,52.145726,16.770268,8.443785,1.025280,1.080822,1.100024,1.014399,1.0000,0.0000,0.035500,0.000000,0.000000,0.000000,EUR,Future
3,2026-02-02,0.9939,51.572437,16.615373,8.266390,1.014008,1.070839,1.076914,0.955288,1.0000,0.0000,0.034755,0.000000,0.000000,0.000000,EUR,Future
4,2026-05-04,0.9891,51.055438,16.704543,8.304638,1.003843,1.076586,1.081897,0.913416,0.7292,0.2708,0.033235,0.270800,0.000000,0.000000,EUR,Future
5,2026-08-03,0.9844,50.511857,16.348817,8.078222,0.993155,1.053660,1.052400,0.870059,0.6722,0.0570,0.022138,0.057000,0.000000,0.000000,EUR,Future
6,2026-11-02,0.9798,49.912689,16.416587,8.117819,0.981374,1.058028,1.057558,0.844874,0.6296,0.0426,0.019355,0.042600,0.000000,0.000000,EUR,Future
7,2027-02-02,0.9752,49.355896,16.247093,7.932394,0.970427,1.047104,1.033402,0.818840,0.6142,0.0154,0.016926,0.015400,0.000000,0.000000,EUR,Future
8,2027-05-03,0.9706,48.737029,16.334889,7.959476,0.958259,1.052762,1.036930,0.797341,0.5976,0.0166,0.015655,0.016600,0.000000,0.000000,EUR,Future
9,2027-08-02,0.9660,48.020758,15.959026,7.726738,0.944175,1.028538,1.006610,0.749782,0.5856,0.0120,0.012290,0.012000,0.000000,0.000000,EUR,Future


##### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_phoenix_memory_basket_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_phoenix_memory_basket_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,8.346580
1,deltaAmountInReportCcy,8.346580
2,gammaAmountInDealCcy,-1.107330
3,gammaAmountInReportCcy,-1.107330
4,vegaAmountInDealCcy,-23.058800
5,vegaAmountInReportCcy,-23.058800
6,thetaAmountInDealCcy,0.090668
7,thetaAmountInReportCcy,0.090668
